In [107]:
import glob 
import os
import re
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import fnmatch
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline
from IPython.core.display import HTML
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [118]:
#Training Data
trainData = os.listdir("TrainingData")[1::]
x_time_files = fnmatch.filter(trainData,'*x_time.csv')
y_time_files = fnmatch.filter(trainData,'*y_time.csv')
x_files = fnmatch.filter(trainData,'*x.csv')
y_files = fnmatch.filter(trainData,'*y.csv')

#Test Data
trainData = os.listdir("TestData")
test_x_time_files = fnmatch.filter(trainData,'*x_time.csv')
test_y_time_files = fnmatch.filter(trainData,'*y_time.csv')
test_x_files = fnmatch.filter(trainData,'*x.csv')

In [3]:
test_y_time_files 

['subject_009_01__y_time.csv',
 'subject_010_01__y_time.csv',
 'subject_011_01__y_time.csv',
 'subject_012_01__y_time.csv']

In [119]:
#Creating Dataframes and joining the grouped filenames
df_x = pd.DataFrame()
df_y = pd.DataFrame()
df_x_time = pd.DataFrame()
df_y_time = pd.DataFrame()
y = pd.DataFrame()
x = pd.DataFrame()
files = sorted(set([x[:14] for x in x_files]))

for file in files:
    temp_x = pd.read_csv(f"TrainingData/{file}__x.csv", names=["accr_x", "accr_y","accr_z","gyr_x","gyr_y","gyr_z"])
    temp_x["subject"] = file
    df_x = df_x.append(temp_x)
    temp_y = pd.read_csv(f"TrainingData/{file}__y.csv",names=["label"])
    temp_y["subject"] = file
    df_y = df_y.append(temp_y)
    temp_xt = pd.read_csv(f"TrainingData/{file}__x_time.csv", names = ["time"])
    df_x_time = df_x_time.append(temp_xt)    
    temp_yt = pd.read_csv(f"TrainingData/{file}__y_time.csv",names=["y_time"])
    df_y_time = df_y_time.append(temp_yt)
    temp_x_concat = pd.concat([temp_xt,temp_x],axis=1)
    x = x.append(temp_x_concat)
    
    temp_y_concat=pd.concat([temp_yt,temp_y],axis=1)
    
    y = y.append(temp_y_concat)
    

In [120]:
y = y.reset_index(drop=True)
x = x.reset_index(drop=True)

In [121]:
labels=[]
# since all y_times and x_times are in order and are in the ratio 1:4. mapping first four x's to first y label and so on.
for i in y["label"]:
    labels += [i]*4
print(len(labels)) #1341652
print(len(x)) #1341646
labels = labels[:1341646]

1341652
1341646


In [122]:
le = preprocessing.LabelEncoder()
x['subject'] = le.fit_transform(x['subject'])


In [123]:
len(x),len(labels)

(1341646, 1341646)

In [125]:
#train_features, test_features, train_labels, test_labels = train_test_split(x, labels, test_size = 0.2, random_state = 42)
train_features, test_features, train_labels, test_labels = x[:1073312], x[1073312:], labels[:1073312], labels[1073312:]
   
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)
rf.fit(train_features, train_labels)

RandomForestClassifier(random_state=42)

In [126]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

print("Accuracy:", accuracy_score(test_labels, predictions))

print("=== Confusion Matrix ===")
print(confusion_matrix(test_labels, predictions))
print("=== Classification Report ===")
print(classification_report(test_labels, predictions))

Accuracy: 0.7148479134213331
=== Confusion Matrix ===
[[182969   2823    910   9096]
 [  3912   2218     29    205]
 [  8605     61   2618    680]
 [ 48466   1305    424   4013]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.75      0.93      0.83    195798
           1       0.35      0.35      0.35      6364
           2       0.66      0.22      0.33     11964
           3       0.29      0.07      0.12     54208

    accuracy                           0.71    268334
   macro avg       0.51      0.39      0.41    268334
weighted avg       0.64      0.71      0.65    268334



In [115]:
len(x), len(labels)

(1341634, 1341646)

In [124]:
n = len(x)
train_features, test_features, train_labels, test_labels = x[:1073312], x[1073312:], labels[:1073312], labels[1073312:]
                                                                            
lr = LogisticRegression(random_state = 42)
lr.fit(train_features, train_labels)
predictions_lr = lr.predict(test_features)

print("Accuracy:", accuracy_score(test_labels, predictions_lr))

print("=== Confusion Matrix ===")
print(confusion_matrix(test_labels, predictions_lr))
print("=== Classification Report ===")
print(classification_report(test_labels, predictions_lr))

Accuracy: 0.7296168208277743
=== Confusion Matrix ===
[[195779     15      4      0]
 [  6362      1      1      0]
 [ 11959      4      1      0]
 [ 54194     14      0      0]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.73      1.00      0.84    195798
           1       0.03      0.00      0.00      6364
           2       0.17      0.00      0.00     11964
           3       0.00      0.00      0.00     54208

    accuracy                           0.73    268334
   macro avg       0.23      0.25      0.21    268334
weighted avg       0.54      0.73      0.62    268334

